In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/its-a-fraud/train.csv
/kaggle/input/its-a-fraud/test.csv
/kaggle/input/its-a-fraud/mock_submission.csv
/kaggle/input/fraudaa/newdf2.csv
/kaggle/input/fraudaa/newtest.csv


In [16]:
df2=pd.read_csv("../input/fraudaa/newdf2.csv")

In [14]:
ntese=pd.read_csv("../input/fraudaa/newtest.csv")
dte=pd.read_csv('../input/its-a-fraud/test.csv')

In [19]:
df2.drop(['Unnamed: 0'],axis=1,inplace=True)
ntese.drop(['Unnamed: 0'],axis=1,inplace=True)

KeyError: "['Unnamed: 0'] not found in axis"

In [18]:
X_test=ntese.copy()

In [6]:
#df2.drop(['TransactionID'],axis=1,inplace=True)
X_test.drop(['TransactionID'],axis=1,inplace=True)

In [7]:
X=df2.drop(['isFraud'],axis=1,inplace=False)
y=df2['isFraud']

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [17]:
#first try with the default paramter

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

clf = RandomForestClassifier()
clf.fit(X_train,y_train)
y_trainpred=clf.predict_proba(X_train)
y_testpred = clf.predict_proba(X_test)

In [19]:
print('train auc with default parameter:',roc_auc_score(y_train,y_trainpred[:,1]))
print('cv auc with default parameter:',roc_auc_score(y_test,y_testpred[:,1]))

train auc with default parameter: 1.0
cv auc with default parameter: 0.939728110606999


In [20]:
tr_auc=[]
te_auc=[]
est=[500,700,900,1000]
dep=[3,5,7,9,11,13]
for i in est:
    for j in dep:
        print('n_estimator_val:',i,'depth_val:',j)
        clf = RandomForestClassifier(n_estimators=i,max_depth=j,class_weight='balanced',n_jobs=-1)
        clf.fit(X_train,y_train)
        y_trainpred=clf.predict_proba(X_train)
        y_testpred = clf.predict_proba(X_test)
        print('train auc:',roc_auc_score(y_train,y_trainpred[:,1]))
        print('cv auc:',roc_auc_score(y_test,y_testpred[:,1]))
        tr_auc.append(roc_auc_score(y_train,y_trainpred[:,1]))
        te_auc.append(roc_auc_score(y_test,y_testpred[:,1]))
        print('-'*10)

For esti: 500 depth: 3
train auc: 0.8236685507026026
cv auc: 0.822916812382168
----------
For esti: 500 depth: 5
train auc: 0.8559510376495738
cv auc: 0.8527251979880353
----------
For esti: 500 depth: 7
train auc: 0.8780273025591356
cv auc: 0.8715074686870036
----------
For esti: 500 depth: 9
train auc: 0.9001325488343841
cv auc: 0.8872321776808458
----------
For esti: 500 depth: 11
train auc: 0.9227314376166583
cv auc: 0.8994834830768892
----------
For esti: 500 depth: 13
train auc: 0.946475758769136
cv auc: 0.9096661439898418
----------
For esti: 700 depth: 3
train auc: 0.8240231544665658
cv auc: 0.8229265657912037
----------
For esti: 700 depth: 5
train auc: 0.8576643404768103
cv auc: 0.8544947254523128
----------
For esti: 700 depth: 7
train auc: 0.8788817793402338
cv auc: 0.8723400458244585
----------
For esti: 700 depth: 9
train auc: 0.8997202208162073
cv auc: 0.886862228027648
----------
For esti: 700 depth: 11
train auc: 0.9218875560831344
cv auc: 0.8990707038087129
----------

In [9]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=1000,max_depth=13,class_weight='balanced',n_jobs=-1)
rf.fit(X,y)

RandomForestClassifier(class_weight='balanced', max_depth=13, n_estimators=1000,
                       n_jobs=-1)

In [20]:
X_test.shape

(147635, 230)

In [21]:
ypred=rf.predict(X_test)

In [22]:
from collections import Counter

Counter(ypred)

Counter({1: 10041, 0: 137594})

In [23]:
ntese['IsFraud']=ypred

In [24]:
neww=ntese[['TransactionID','IsFraud']]

In [25]:
dte = dte.merge(neww, how='outer',copy=False, on ='TransactionID' )

In [26]:
dte['IsFraud']

0         1
1         1
2         0
3         0
4         0
         ..
147630    0
147631    0
147632    0
147633    0
147634    0
Name: IsFraud, Length: 147635, dtype: int64

In [27]:
y_predd=dte['IsFraud']

In [28]:
Counter(y_predd)

Counter({1: 10041, 0: 137594})

In [29]:
y_predd.to_csv('y_predRF.csv')

In [37]:
output = pd.DataFrame({

"isFraud": y_predd
})

output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
